In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import warnings

warnings.filterwarnings('ignore')

In [2]:
# Метрики расстояний

def euclidean(X, value):
    
    distances = np.linalg.norm(X - value, axis=1)
    return distances

def cosine(cos_matrix):
    
    distances = cos_matrix[df.loc[df.hash_inn == inn].index[0]]
    return -distances

In [3]:
def percentile(p, distances, X):
    
        return np.round(np.mean(np.sort(distances)[:len(X)*p//100]),5)
    
def top_n(n, distances, X):
    
        return np.round(np.mean(np.sort(distances)[:n]),5)

def similars(X, value, inn, N = 5, treshold = 1, metric = euclidean, matrix = None):

    if metric == euclidean:
        distances = metric(X, value)
    elif metric == cosine:
        if matrix is None:
            return print('Нужно передать матрицу cos_matrix')
        else:
            distances = metric(matrix)
        
    arg_nearest = distances.argsort()[0:N]
    
    n_dist = np.sort(distances)[0:N]
    arg_nearest = arg_nearest[(n_dist < treshold)]
    
    #отношение дистанции относительно ближайшего элемента
    per_dist = [abs(n_dist[i]/n_dist[0]) if n_dist[0] != 0 else abs(n_dist[i]) for i in range(len(n_dist))]
    
    result = df.iloc[arg_nearest]
    result['distance'] = abs(n_dist)
    
    
    print('mean distance in percentile: \n p1: {0} \n p5: {1} \n p10: {2} \n p25: {3} \n'
                  .format(*[abs(percentile(p, distances, X)) for p in [1,5,10,20]]))
    
    print('mean distance in top N values: \n top5: {0} \n top10: {1} \n top20: {2} \n top50: {3} \n'
                  .format(*[abs(top_n(p, distances, X)) for p in [5,10,20,50]]))
    
    print('отношение первых N дистанций к дистанции ближайшего элемента: \n', per_dist, '\n')
    print('\n', result[['hash_inn','okved2','distance']])
    print('\n real okved =', df.loc[df.hash_inn==inn].okved2.values[0])
    
    return result[['hash_inn','okved2','distance']]

In [4]:
"""
X - выходной эмбеддинг
result_df - датафрейм размерности inn, okved + len(X) и тем же порядком inn
"""
result_df = pd.read_csv('../data/result_df.csv').fillna(0)
result_df = result_df.iloc[:100]

# в качестве эмбеддинга возьмем сами значения датафрейма
X = result_df.filter(regex=("count_kt")).values

df = pd.concat([result_df[['hash_inn','okved2']],
                pd.DataFrame(X, columns= [str(i) for i in range(len(X[0]))])], axis=1)

In [5]:
# вычисляем косинусное расстояние
cos_matrix = cosine_similarity(X)

In [6]:
"""
metric - реализованные метрики расстояний (cosine, euclidean)
matrix - матрица косинусных расстояний 
"""
inn = 61058
result = similars(X, X[df.loc[df.hash_inn == inn].index[0]], inn, N = 15, treshold = 100,
                  metric = euclidean, matrix = None)

mean distance in percentile: 
 p1: 0.0 
 p5: 0.0 
 p10: 0.0 
 p25: 0.0 

mean distance in top N values: 
 top5: 0.0 
 top10: 0.0 
 top20: 0.0 
 top50: 0.49047 

отношение первых N дистанций к дистанции ближайшего элемента: 
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 


     hash_inn  okved2  distance
0      61058      34       0.0
24    221180      52       0.0
25    123045      26       0.0
27    242879      34       0.0
29    154902      18       0.0
34    174903      14       0.0
35      2285      16       0.0
36      9463      11       0.0
40     49299       6       0.0
48    256315      34       0.0
51    241257      67       0.0
57    116503      52       0.0
60    117458      14       0.0
63    156771      26       0.0
67     43234      23       0.0

 real okved = 34
